<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/ninologo.png" width="128"/>
<img align="right" src="images/dans.png" width="128"/>

---

To get started: consult [start](start.ipynb)

---

# Display

We show the ins and outs of displaying cuneiform ATF transcriptions.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.app import use

In [73]:
A = use("Nino-cunei/ninmed:clone", checkout="clone", hoist=globals())
# A = use("Nino-cunei/ninmed", hoist=globals())

This is Text-Fabric 9.2.5
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

45 features found and 0 ignored


We pick an example face with which we illustrate many ways to represent cuneiform text.

In [63]:
exampleFace = ("P479250", "reverse")
f = T.nodeFromSection(exampleFace)
lines = L.d(f, otype="line")

# Raw text

The most basic way is to show the source material for each line by its `atf` feature.
As this does not include the line number, we grab that from the `lnno` feature.

In [64]:
for ln in lines:
    print(F.lnno.v(ln), F.atf.v(ln))

!1  
1 1. %sux [...] ka#-keš₂#-bi# he₂-du₈-a
2 2. %sux [...] ka-keš₂-bi he₂-du₈-a
3 3. [x x x x x x x x x x x x ina KA-š]u₂ u₂-man-zaq-ma ina-eš
4 4. [... S]IG₃.SIG₃-ta ma-la-a rit-ta-šu₂
5 5. [... a]-ha-tu mu-še-qa-at re-ši-šu₂
6 6. [... na-š]i pat-ri TU₆ EN₂
7 7. [x x x x x x x x x x x ana UG]U EN₂ ŠUB-di ina KA-šu₂ u₂-man-zaq-ma ina-eš
8 8. [x x x x x x x x x x x x as-su]h-šu₂-ma iṣ-ṣa-bat ŠA₃-bi
9 9. [x x x x x x x x x x x x x x x x x x K]I DU₁₁.GA-ma iṣ-ṣa-bat ŠA₃-bi AN u KI
10 10. [x x x x x x x x x x x x x x x x a-na u₂-ma]-mi# DU₁₁.GA-ma iṣ-ṣa-bat ŠA₃-bi u₂-ma-me
11 11. [x x x x x x x x x x x x x Š]A₃-bi {d}UTU lip-pa-šir
12 12. [x x x x x x x Š]A₃-bi# u₂-ma-me lip-pa-šir
13 13. [x x x x x x x x x x x x šam-ma]-ka NAG-šu₂ lib-luṭ
14 14. [...] U₂# KI KI MIN KI MIN
15 15. [...] ŠA₃#-bu i-re-du-šu
16 16. [... ap-p]i? šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂
17 17. [... SAHA]R? TA# {urudu#}na#-ah#-bi?# [ana? {urud]u}na#-ah#-bi?#


Note that the document starts with an empty line, which is not ATF-numbered.
Here the feature `lnno` contains `!` followed by the logical line number on the face.

# Text formats

The TF API supports *text formats*. Text formats make selections and apply templates and styles based
on the analysed features of the text. For example: a text-format may ignore flags or clusters, or
format numerals in special ways.

Text formats are not baked into TF, but they are defined in the feature `otext` of the corpus.

Moreover, for this corpus a TF app has been build that defines additional text-formats.

Whereas the formats defined in `otext` are strictly plain text formats, the formats
defined in the app are able to use typographic styles to shape the text, such as bold, italic, colors, etc.

Here is the list of all formats.

In [65]:
T.formats

{'text-orig-full': 'sign',
 'text-orig-plain': 'sign',
 'layout-orig-full': 'sign',
 'layout-orig-plain': 'sign'}

## Plain text formats

The formats whose names start with `text-` are the plain text formats.

### `text-orig-full`

This format is really close to the ATF. It contains all original information.

This is the default format. We do not have to specify it.

In [66]:
for ln in lines:
    print(F.lnno.v(ln), T.text(ln))

!1 
1 %sux [...] ka#-keš₂#-bi# he₂-du₈-a 
2 %sux [...] ka-keš₂-bi he₂-du₈-a 
3 [x x x x x x x x x x x x ina KA-] šu₂# u₂-man-zaq-ma ina-eš 
4 [... ] SIG₃#.SIG₃-ta ma-la-a rit-ta-šu₂ 
5 [... a]-ha-tu mu-še-qa-at re-ši-šu₂ 
6 [... na-] ši# pat-ri TU₆ EN₂ 
7 [x x x x x x x x x x x ana ] UGU# EN₂ ŠUB-di ina KA-šu₂ u₂-man-zaq-ma ina-eš 
8 [x x x x x x x x x x x x as-] suh#-šu₂-ma iṣ-ṣa-bat ŠA₃-bi 
9 [x x x x x x x x x x x x x x x x x x ] KI# DU₁₁.GA-ma iṣ-ṣa-bat ŠA₃-bi AN u KI 
10 [x x x x x x x x x x x x x x x x a-na u₂-ma]-mi# DU₁₁.GA-ma iṣ-ṣa-bat ŠA₃-bi u₂-ma-me 
11 [x x x x x x x x x x x x x ] ŠA₃#-bi dUTU lip-pa-šir 
12 [x x x x x x x ] ŠA₃#-bi# u₂-ma-me lip-pa-šir 
13 [x x x x x x x x x x x x šam-ma]-ka NAG-šu₂ lib-luṭ 
14 [...] U₂# KI KI MIN KI MIN 
15 [...] ŠA₃#-bu i-re-du-šu 
16 [... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂ 
17 [... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?# 


The `A.plain()` function focuses on the *contents*, and instead of the line number, it gives a full specification
of the location, linked to the online source on CDLI.

In [67]:
for ln in lines:
    A.plain(ln)

### `text-orig-plain`

This is a somewhat reduced format. It omits all flags and bracketing constructs.

In [68]:
for ln in lines:
    A.plain(ln, fmt="text-orig-plain")

## Styled text formats

The formats whose names start with `layout-` are the styled text formats.

### `layout-orig-full`

This format looks like `text-orig-full`, but now we re-introduce the flags and clusters by specific
layout devices.

See below for detailed examples.

In [74]:
for ln in lines:
    A.plain(ln, fmt="layout-orig-full")

### `layout-orig-plain`

This format looks like `text-orig-plain`, but now we re-introduce the flags and clusters by specific
layout devices.

See below for detailed examples.

In [75]:
for ln in lines:
    A.plain(ln, fmt="layout-orig-plain")

Here is the text of the face in each of the plain text formats, i.e. no additional HTML formatting is applied.

# Pretty

The ultimate of graphical display is by means of the `pretty()` function.

This display is less useful for reading, but instead optimized for showing all information that you might
wish for.

It shows a base representation according to a text format of your choice
(here we choose `layout-orig-full`), and it shows the values
of a standard set of features.

In [78]:
w = F.otype.s("word")[10]
F.atf.v(w)

'hur.ri.im#'

In [79]:
A.pretty(w)

In [80]:
A.pretty(w, fmt="layout-orig-full", withNodes=True)

By default, pretty displays descend to the word level, but you can also descend to the sign level:

In [81]:
A.pretty(w, baseTypes="sign")

In [82]:
A.pretty(w, fmt="layout-orig-full", baseTypes="sign", withNodes=True)

Later on, in the [search](search.ipynb) tutorial we see that `pretty()` can also display other features,
even features that you or other people have created and added later.

Here we call for the feature `atf`, which shows the original atf for the sign in question
excluding the bracketing characters.

Consult the
[feature documentation](https://github.com/Nino-cunei/oldbabylonian/blob/master/docs//transcription.md)
to see what information is stored in all the features.

We show it with node numbers, but you could leave them out in an obvious way.

In [86]:
A.pretty(f, extraFeatures="atf", fmt="layout-orig-full", withNodes=True)

We do not see much, because the default condense type is `line`, and a `document` is bigger than that.
Objects bigger than de condense type will be abbreviated to a label that indicates their identity,
not their contents.

But we can override this by adding `full=True`.

See also the documentation on [`pretty`](https://annotation.github.io/text-fabric/tf/advanced/display.html#tf.advanced.display.pretty).

In [88]:
A.pretty(f, extraFeatures="atf", fmt="layout-orig-plain", withNodes=True, full=True)

# Layout formats: the details

We give detailed examples of how the material is styled in the `layout-` formats.

We show the representation of all kinds of signs and also what the influence of
clustering and flags are.

Here are the design principles:

* all flags `# ? ! *` cause the preceding sign to be in bold
* damage `#` and missing `[ ]` text is blurry and in grey
* questioned `?` and uncertain `( )` text is in italics
* remarkable `!` and supplied `< >` text is overlined, supplied text is in blue
* excised `<< >>` text has a strike-through and is in red
* collated `*` text is underlined

Just a quick overview of the sign types:

In [89]:
F.type.freqList("sign")

(('grapheme', 20433),
 ('reading', 20357),
 ('joiner', 18957),
 ('mark', 12454),
 ('unknown', 4720),
 ('ellipsis', 1236),
 ('numeral', 958),
 ('lang', 186),
 ('wdiv', 92),
 ('empty', 69),
 ('erasure', 26))

# Styled display of ATF text

In [93]:
lines = (
    (("P365742", "reverse", "1:15"), ("cluster: determinative", [9, 10])),
    (("P509375", "obverse", "3"), ("cluster: missing", [1, 2, 3, 6])),
    (("P510736", "reverse", "5"), ("cluster: uncertain", [6, 7, 8])),
    (("P510526", "obverse", "8"), ("cluster: supplied", [7, 8, 9])),
    (("P373056", "obverse", "11"), ("cluster: excised", [9, 10])),
    (("P510536", "obverse", "8"), ("flag: damage", [1, 2, 3, 6, 9, 10])),
    (("P297171", "reverse", "1"), ("flag: question", [3, 4, 5])),
    (("P510536", "obverse", "15"), ("flag: remarkable", [6])),
    (("P387299", "obverse", "5"), ("flag: collated", [1])),
    (("P510534", "reverse", "20'"), ("flag: damage + question", [3])),
    (("P497780", "reverse", "4"), ("flag: damage + remarkable", [1])),
    (("P305744", "obverse", "10"), ("flag: damage + collated", [5])),
    (("P305799", "left side", "1"), ("flag: collated + remarkable", [3, 5])),
    (("P305806", "reverse", "1"), ("flag: remarkable + question + collated", [3])),
    (("P305744", "obverse", "10"), ("flag collated in cluster missing", [6, 7])),
    (("P275088", "reverse", "2"), ("sign: comment", [1])),
    (("P510636", "obverse", "3"), ("sign: grapheme", [9])),
    (("P386451", "reverse", "11"), ("sign: correction", [6])),
    (("P312032", "left edge", "1"), ("sign: numeral", [4, 6])),
    (("P492284", "obverse", "5"), ("sign: ligature", [2, 3])),
    (("P509373", "reverse", "18'"), ("sign: unknown and ellipsis", [7, 8])),
)

In [102]:
for (line, (desc, positions)) in lines:
    ln = T.nodeFromSection(line)
    A.dm("---\n# {}\n\nLocation: {} {}:{}".format(desc, *line))
    s = L.d(ln, otype="sign")[0]
    highlights = {s + p - 1 for p in positions}
    print(F.lnno.v(ln), T.text(ln))
    A.plain(ln, fmt="layout-orig-full", highlights=highlights)
    A.pretty(
        ln,
        extraFeatures="tr@en",
        fmt="text-orig-full",
        baseTypes="sign",
        highlights=highlights,
    )
    break

---
# cluster: determinative

Location: P365742 reverse:1:15

1:15 [ana KI ] MIN# šimGUR₂.GUR₂ šimLI šim.dNIN.URTA NUMUN u₂AB₂.DUH KA A.AB.BA šimŠEŠ DIŠ-niš SUD₂ ina KAŠ SILA₁₁-aš SAR-ab KI MIN 


---

All chapters:

* **[start](start.ipynb)** introduction to computing with your corpus
* **display** become an expert in creating pretty displays of your text structures
* **[search](search.ipynb)** turbo charge your hand-coding with search templates
* **[exportExcel](exportExcel.ipynb)** make tailor-made spreadsheets out of your results
* **[share](share.ipynb)** draw in other people's data and let them use yours
* **[similarLines](similarLines.ipynb)** spot the similarities between lines

---

See the [cookbook](cookbook) for recipes for small, concrete tasks.

CC-BY Dirk Roorda